In [ ]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 1.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357243 sha256=b003a5311aa446997a93b81c3632c25b36500240d8695314aa770903c3d0ff9f
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
import pandas as pd
import numpy as np
import surprise
import os

In [ ]:
os.chdir("/content/drive/MyDrive/Cases/filmtrust")
ratings = pd.read_csv("ratings.txt",sep=' ',names = ['uid','iid','rating'])
ratings.head()

,uid,iid,rating
0,1,1,2.0
1,1,2,4.0
2,1,3,3.5
3,1,4,3.0
4,1,5,4.0


In [ ]:
lowest_rating = ratings['rating'].min()
highest_rating = ratings['rating'].max()
print("Ratings range between {0} and {1}".format(lowest_rating,highest_rating))

Ratings range between 0.5 and 4.0


In [ ]:
reader = surprise.Reader(rating_scale = (lowest_rating,highest_rating))
data = surprise.Dataset.load_from_df(ratings,reader)
type(data)

surprise.dataset.DatasetAutoFolds

Tuning for best parameters

In [ ]:
from surprise.model_selection import GridSearchCV
from surprise.model_selection.split import KFold

param_grid = {'lr_all':np.linspace(0.001,1,3), 'reg_all': np.linspace(0.01,0.8,3),
              'n_factors':[40,30]}
kfold = KFold(random_state=23,n_splits=5,shuffle=True)
gs = GridSearchCV(surprise.SVD, param_grid,joblib_verbose =3, measures=['rmse', 'mae'],
                  cv=kfold,n_jobs=-1)
gs.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   14.2s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:   36.4s finished


Best Score:

In [ ]:
print(gs.best_score['rmse'])

0.8284872423422218


Best Parameter:

In [ ]:
print(gs.best_params['rmse'])

{'lr_all': 0.001, 'reg_all': 0.01, 'n_factors': 30}


We can now use the algorithm that yields the best rmse:

In [ ]:
algo = gs.best_estimator['rmse']
algo.fit(data.build_full_trainset())

The recommendations can be generated for any user with the object **algo**.

Total Items:

In [ ]:
iids = ratings['iid'].unique()
print(iids)

[   1    2    3 ... 2069 2070 2071]


The list of items rated by user 50:

In [ ]:
u_iid = ratings[ratings['uid']==50]['iid'].unique()
print(u_iid)

[  8 211   3   2 219 234  12 254 250 207  11 253 236  84  10   7 233  13
   1   5   6 252 241 216 257 206   4 217   9 215 213  17 255 220 121 245
 239 251 235]


List of the items not rated by user 50:

In [ ]:
iids_to_predict = np.setdiff1d(iids, u_iid)
print(iids_to_predict)

[  14   15   16 ... 2069 2070 2071]


Extracting the estimated rating from iids_to_predict

In [ ]:
testset = [[50,iid,0.] for iid in iids_to_predict]
predictions = algo.test(testset)

In [ ]:
predictions[0]

Prediction(uid=50, iid=14, r_ui=0.0, est=3.1820403487880315, details={'was_impossible': False})

In [ ]:
pred_ratings = np.array([pred.est for pred in predictions])

Getting the item with highest expected rating

In [ ]:
i_max = np.argmax(pred_ratings)
print("Item:",iids_to_predict[i_max],"is the item with highest expected rating as",pred_ratings[i_max])

Item: 218 is the item with highest expected rating as 3.6767261551144728


In [ ]:
exp_ratings = pd.DataFrame({'Item_ID': iids_to_predict, 'Exp_Rating':pred_ratings})
exp_ratings.sort_values(by=['Exp_Rating','Item_ID'], ascending=[False, True], inplace=True)
exp_ratings.head(10)

,Item_ID,Exp_Rating
194,218,3.676726
210,242,3.616111
278,318,3.595095
635,675,3.584358
765,805,3.554460
307,347,3.547540
246,286,3.546373
688,728,3.518231
325,365,3.514065
392,432,3.513155
